In [66]:
#Read the base bom files from jde as dataframes

import pandas as pd
import os


outputfolder = r'C:\Users\SPoudel\OneDrive - Ventura Foods\Desktop\Python sample outputs'

# Load the Excel file without headers
jde_bom = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/jde_us.xlsx", 
    skiprows=2
)

'''
jde_bom_can = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/jde_canada.xlsx", 
    skiprows=2
)
'''

# this completes the horizontals(rows) of data creating needed base Records or Observations
#jde_bom = pd.concat([jde_bom_us,jde_bom_can],axis=0,join='outer') #no need for canada separate pull now
jde_bom.columns = jde_bom.columns.str.replace(" ", "_", regex=True) #replace spaces and blanks with "_"
jde_bom = jde_bom.drop(columns=["3rd_Item_Number", "Extended_Amount"])

jde_bom["Campus_SKU"] = jde_bom["Business_Unit"].astype(str) + "_" + jde_bom["Parent_Item_Number"].astype(str) #creating ref by combining Business Unit and Parent Item Number
jde_bom["Campus_Component"] = jde_bom["Business_Unit"].astype(str) + "_" + jde_bom["Component"].astype(str) #creating component ref by combining business unit and component
jde_bom.columns = ["Campus","Level","SKU","Parent_SKU_Description","UOM","Component","Component_Description",
                   "Commodity_Class","UM","Stocking_Type","Percent_Scrap","Quantity_Per","Quantity_w_Scrap","Unit_Cost","Campus_SKU","Campus_Component"] #Changing column names to appropriate names

In [67]:
#Sales Order in next 28 days attribute design with location and item# as key
#MODIFICATION: THIS CAN BE MODIFIED WITH NEW DATASET FOR NEXT 28 DAYS OPEN ORDERS AND NO NEED TO GET CANADA OPEN ORDERS
custorder28days = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/US and CAN OO BT where status _ J.xlsx",skiprows=2)
custorder28days['Product Label (SKU) Code'].replace("-","",regex=True,inplace=True)
#custorder28days['Location_SKU'] = custorder28days['Location No'].astype(str) + '_' + custorder28days['Product Label (SKU) Code'].astype(str)
#custorder28days['Campus_SKU'] = custorder28days['Campus No.'].astype(str) + '_' + custorder28days['Product Label (SKU) Code'].astype(str)
custorder28days = custorder28days.iloc[1:]
#custorder28days.to_excel(os.path.join(outputfolder,"custord28days.xlsx"), index=False)
custorder28days['TotalOO'] = custorder28days['OO Cases'].fillna(0) + custorder28days['B/T Open Order Cases'].fillna(0)
custord28days = custorder28days.groupby(['Campus No.','Product Label (SKU) Code'])['TotalOO'].sum().reset_index()
custord28days.columns = ['Campus','SKU','TotalOO']

C:\Users\SPoudel\AppData\Local\Temp\ipykernel_27800\2124428609.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  custorder28days['Product Label (SKU) Code'].replace("-","",regex=True,inplace=True)


In [68]:
#Forecast Dataset
import numpy as np

ForecastLatestCopy= pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/ForecastThisNext&6Months.xlsx",skiprows=2)

ForecastLatestCopy['Campus'] = np.where(ForecastLatestCopy['Product Manufacturing Location Code'] == -1,ForecastLatestCopy['Location No'],ForecastLatestCopy['Product Manufacturing Location Code'])
ForecastLatestCopy['Product Label (SKU) Code'].replace("-","",regex=True,inplace=True)
ForecastFinal = ForecastLatestCopy.groupby(['Campus','Product Label (SKU) Code'])[['Forecast This Month','Forecast Next Month','Forecast 6 Months']].sum().reset_index()
ForecastFinal.columns = ['Campus','SKU','CurrentMonthForecast','NextMonthForecast','Next6MonthForecast']

c:\Anaconda_Distribution\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\SPoudel\AppData\Local\Temp\ipykernel_27800\1235566523.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ForecastLatestCopy['Product Label (SKU) Code'].replace("-","",regex=True,inplace=True)


In [ ]:
bom = jde_bom.merge(custord28days,how='left',on=['Campus','SKU']).merge(ForecastFinal,how='left',on=['Campus','SKU'])

bom['CurrentMthDepDemand'] = bom['CurrentMonthForecast']* bom['Quantity_w_Scrap']
bom['NextMthDepDemand'] = bom['NextMonthForecast']* bom['Quantity_w_Scrap']
bom['Next6MthDepDemand'] = bom['Next6MonthForecast']* bom['Quantity_w_Scrap']


In [76]:
bom.to_excel(r'C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/bom.xlsx')

In [77]:
bom

,Campus,Level,SKU,Parent_SKU_Description,UOM,Component,Component_Description,Commodity_Class,UM,Stocking_Type,...,Unit_Cost,Campus_SKU,Campus_Component,TotalOO,CurrentMonthForecast,NextMonthForecast,Next6MonthForecast,CurrentMthDepDemand,NextMthDepDemand,Next6MthDepDemand
0,10,1,12277BKO,CNO RBD BC 40R NTF TT,LB,5114,"BETA CAROTENE,LIQ,22%,HEATSTBL",320,LB,P,...,27.6390,10_12277BKO,10_5114,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,1,12277BKO,CNO RBD BC 40R NTF TT,LB,9513,"RBD, CNO",SFM,LB,M,...,1.2368,10_12277BKO,10_9513,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,2,12277BKO,CNO RBD BC 40R NTF TT,LB,9512,"RB, CNO",SFM,LB,M,...,1.2175,10_12277BKO,10_9512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,3,12277BKO,CNO RBD BC 40R NTF TT,LB,8903,CAUSTIC SODA 50% Food Grade,RPS,LB,P,...,0.3590,10_12277BKO,10_8903,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,3,12277BKO,CNO RBD BC 40R NTF TT,LB,8910,FILTER AID FW 14,RPS,LB,P,...,1.7128,10_12277BKO,10_8910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64250,624,3,61420SYC,TARTAR SYSCO IMP 2/3.78L,CS,390000133,"EGG,WHOLE,FRZ 00030 BRNB",170,KG,P,...,5.4600,624_61420SYC,624_390000133,241.0,350.0,280.0,1750.0,51.85915,41.48732,259.29575
64251,624,3,61420SYC,TARTAR SYSCO IMP 2/3.78L,CS,390000132,"EGG,YOLK,SLT,00098 BRNB FRZ",170,KG,P,...,5.4800,624_61420SYC,624_390000132,241.0,350.0,280.0,1750.0,57.62225,46.09780,288.11125
64252,624,3,61420SYC,TARTAR SYSCO IMP 2/3.78L,CS,360010007,"SUGAR,LIQUID,SUCROSE",280,KG,P,...,0.8184,624_61420SYC,624_360010007,241.0,350.0,280.0,1750.0,61.08865,48.87092,305.44325
64253,624,2,61420SYC,TARTAR SYSCO IMP 2/3.78L,CS,320000093,"RELISH, DILL, 6870",300,KG,P,...,1.9787,624_61420SYC,624_320000093,241.0,350.0,280.0,1750.0,207.13770,165.71016,1035.68850


OLD Code for Old OLD Ways of getting data

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

#Forecast Dataset
ForecastLatestCopy= pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/DSX Forecast Backup - 2025.03.07.xlsx",skiprows=2)
ForecastLatestCopy['Forecast Month Year ID'] = pd.to_datetime(ForecastLatestCopy["Forecast Month Year ID"], format="%Y%m") + pd.offsets.MonthBegin(0)

today = pd.to_datetime("today")
ForecastLatestCopy['MonthFromCurrent'] = (ForecastLatestCopy['Forecast Month Year ID'].dt.year - today.year)*12 + (ForecastLatestCopy['Forecast Month Year ID'].dt.month - today.month)
ForecastLatestCopy['Product Label (SKU) Code'] = ForecastLatestCopy['Product Label (SKU) Code'].str.replace('-','') #change SKU name and without - in middle
ForecastLatestCopy['Campus_SKU'] = ForecastLatestCopy['Product Manufacturing Location Code'].astype(str) + "_" + ForecastLatestCopy['Product Label (SKU) Code'].astype(str) #Create Key (manf. Loc. acts as Campus and SKU) to join with other dataframes
#ForecastLatestCopy['Adjusted Forecast Cases'] = ForecastLatestCopy['Adjusted Forecast Cases'].fillna(0)

FcstNxt2Mths = ForecastLatestCopy[(ForecastLatestCopy['MonthFromCurrent'] < 2) & (ForecastLatestCopy['MonthFromCurrent'] >= 0)]
FcstNext6Months = ForecastLatestCopy[(ForecastLatestCopy['MonthFromCurrent'] < 6) & (ForecastLatestCopy['MonthFromCurrent'] >= 0)]
FcstNext12Months = ForecastLatestCopy[(ForecastLatestCopy['MonthFromCurrent'] < 12) & (ForecastLatestCopy['MonthFromCurrent'] >= 0)]

FcstNxt2MthsPivot =  FcstNxt2Mths.pivot_table(values='Adjusted Forecast Cases',index='Campus_SKU',columns='MonthFromCurrent',aggfunc='sum').reset_index()
FcstNxt2MthsPivot.rename(columns={0:'ThisMonthFcst',1:'NextMonthFcst'}, inplace=True)

FcstNext6MonthsPivot =  FcstNext6Months.pivot_table(values='Adjusted Forecast Cases',index='Campus_SKU',aggfunc='sum').reset_index()
FcstNext6MonthsPivot.rename(columns={'Adjusted Forecast Cases':'Next6MonthsFcst'}, inplace=True)
FcstCondensed =   FcstNxt2MthsPivot.merge(FcstNext6MonthsPivot,how='outer')

In [ ]:
#Inventory With Hold Status for FG and RM
InvWHoldStatus= pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/InventoryWCampusForBOM_FROMObject.xlsx",skiprows=2)

#InvWHoldStatus.columns = ['Campus','Location','SKU','Item_Type','Hard_Hold','Soft_Hold','Useable']
InvWHoldStatus_FG = InvWHoldStatus[InvWHoldStatus['Item_Type'] == 'FG']
InvWHoldStatus_RM = InvWHoldStatus[InvWHoldStatus['Item_Type'] == 'COMP']
InvWHoldStatus_WIP = InvWHoldStatus[InvWHoldStatus['Item_Type'] == 'WIP']


LabelInventory25 = InvWHoldStatus= pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Current Week Dataset_test/JDE Inventory Lot Detail.xlsx",skiprows=6)
LabelInventory25 = LabelInventory25[(LabelInventory25['MPF'] == 'LBL') & (LabelInventory25['BP'] == 25)][['BP','Item Number','Status','On Hand']]
LabelInventory25

#reading Lot status Lookup
LotStatusCode = pd.read_excel(
    "C:/Users/SPoudel/Ventura Foods/SC Analytics Team - General/Stan Report Files/File Repository/Lookup Datasets/Static_MonthlyLookup Dataset/Lot Status Code.xlsx")
LabelInventory25WHoldStatus =  LabelInventory25.merge(LotStatusCode[['Lot status','Hard/Soft Hold']],how='left',left_on='Status',right_on='Lot status')[['BP','Item Number','On Hand','Hard/Soft Hold']]
LabelInventory25WHoldStatus['Hard/Soft Hold'] =LabelInventory25WHoldStatus['Hard/Soft Hold'].fillna('Useable')